## Statsmodel Forecast with Wallaroo Features: ML Workload Orchestration

Wallaroo provides Data Connections and ML Workload Orchestrations to provide organizations with a method of creating and managing automated tasks that can either be run on demand or a regular schedule.

## Prerequisites

* A Wallaroo instance version 2023.2.1 or greater.

## References

* [Wallaroo SDK Essentials Guide: Model Uploads and Registrations: Python Models](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/wallaroo-sdk-model-upload-python/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)
* [Wallaroo SDK Essentials Guide: ML Workload Orchestration](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-ml-workload-orchestration/)

## Orchestrations, Taks, and Tasks Runs

We've details how Wallaroo Connections work.  Now we'll use Orchestrations, Tasks, and Task Runs.

| Item | Description |
|---|---|
| Orchestration | ML Workload orchestration allows data scientists and ML Engineers to automate and scale production ML workflows in Wallaroo to ensure a tight feedback loop and continuous tuning of models from training to production. Wallaroo platform users (data scientists or ML Engineers) have the ability to deploy, automate and scale recurring batch production ML workloads that can ingest data from predefined data sources to run inferences in Wallaroo, chain pipelines, and send inference results to predefined destinations to analyze model insights and assess business outcomes. |
| Task | An implementation of an Orchestration.  Tasks can be either `Run Once`:  They run once and upon completion, stop. `Run Scheduled`: The task runs whenever a specific `cron` like schedule is reached.  Scheduled tasks will run until the `kill` command is issued. |
| Task Run | The execusion of a task.  For `Run Once` tasks, there will be only one `Run Task`.  A `Run Scheduled` tasks will have multiple tasks, one for every time the schedule parameter is met.  Task Runs have their own log files that can be examined to track progress and results. |

## Statsmodel Forecast Connection Steps

### Import Libraries

The first step is to import the libraries that we will need.

In [12]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
import numpy as np

from resources import simdb
from resources import util

pd.set_option('display.max_colwidth', None)

import time

In [13]:
display(wallaroo.__version__)

'2023.2.1'

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [14]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

wallarooPrefix = "doc-test."
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Set Configurations

The following will set the workspace, model name, and pipeline that will be used for this example.  If the workspace or pipeline already exist, then they will assigned for use in this example.  If they do not exist, they will be created based on the names listed below.

Workspace names must be unique.  To allow this tutorial to run in the same Wallaroo instance for multiple users, set the `suffix` variable or share the workspace with other users.

#### Set Configurations References

* [Wallaroo SDK Essentials Guide: Workspace Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-workspace/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)

In [15]:
# used for unique connection names

import string
import random

suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))

suffix='jch'

workspace_name = f'forecast-model-workshop{suffix}'

pipeline_name = 'forecast-workshop-pipeline'

### Set the Workspace and Pipeline

The workspace will be either used or created if it does not exist, along with the pipeline.  The models uploaded in the Upload and Deploy tutorial are referenced in this step.

In [16]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

# Get the most recent version of a model in the workspace
# Assumes that the most recent version is the first in the list of versions.
# wl.get_current_workspace().models() returns a list of models in the current workspace

def get_model(mname):
    modellist = wl.get_current_workspace().models()
    model = [m.versions()[0] for m in modellist if m.name() == mname]
    if len(model) <= 0:
        raise KeyError(f"model {mname} not found in this workspace")
    return model[0]

# upload three models:  the control and two challengers

control_model_name = 'forecast-control-model'
challenger01_model_name = 'forecast-challenger01-model'
challenger02_model_name = 'forecast-challenger02-model'

# retrieve the models

bike_day_model = get_model(control_model_name)

challenger_model_01 = get_model(challenger01_model_name)

challenger_model_02 = get_model(challenger02_model_name)

### Deploy Pipeline

The pipeline is already set witht the model.  For our demo we'll verify that it's deployed.

In [17]:
# Set the deployment to allow for additional engines to run
# Undeploy and clear the pipeline in case it was used in other demonstrations
pipeline.undeploy()
pipeline.clear()
deploy_config = (wallaroo.DeploymentConfigBuilder()
                        .replica_count(1)
                        .replica_autoscale_min_max(minimum=2, maximum=5)
                        .cpus(0.25)
                        .memory("512Mi")
                        .build()
                    )

pipeline.add_model_step(bike_day_model)
# pipeline.add_model_step(step)
pipeline.deploy(deployment_config = deploy_config)

{'name': 'forecast-workshop-pipeline', 'create_time': datetime.datetime(2023, 7, 27, 15, 54, 55, 416132, tzinfo=tzutc()), 'definition': "[{'ModelInference': {'models': [{'name': 'forecast-control-model', 'version': '0daa07c7-797d-429f-8a3b-7bba4f709da0', 'sha': '60156c8e9bad5b9fee1dfea2aeae05ca3f643f53b5b6d6a365e8e020c72af6ac'}]}}]"}

### Forecast Sample Orchestration

The orchestration that will automate this process is `./resources/forecast-orchestration.zip`.  The files used are stored in the directory `forecast-orchestration`, created with the command:

`zip -r forecast-bigquery-connection.zip forecast-orchestration/`.

This contains the following:

* `requirements.txt`:  The Python requirements file to specify the following libraries used.  For this example, that will be empty since we will be using the 
* `main.py`: The entry file that uses a deployed pipeline and performs an inference request against it visible from its log files.
* `data/testdata_dict.json`: An inference input file.

The `main.py` script performs a workspace and pipeline retrieval, then an inference against the inference input file.

```python
import json
import os


import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)

wl = wallaroo.Client()

# get the arguments
arguments = wl.task_args()

if "workspace_name" in arguments:
    workspace_name = arguments['workspace_name']
else:
    workspace_name="multiple-replica-forecast-tutorial"

if "pipeline_name" in arguments:
    pipeline_name = arguments['pipeline_name']
else:
    pipeline_name="bikedaypipe"

if "bigquery_connection_input_name" in arguments:
    bigquery_connection_name = arguments['bigquery_connection_input_name']
else:
    bigquery_connection_name = "statsmodel-bike-rentals"

print(bigquery_connection_name)
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        print(f"Pipeline not found:{name}")
    return pipeline

print(f"Workspace: {workspace_name}")
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)
print(workspace)

# the pipeline is assumed to be deployed
print(f"Pipeline: {pipeline_name}")
pipeline = get_pipeline(pipeline_name)
print(pipeline)

inferencedata = json.load(open("./data/testdata_dict.json"))

results = pipeline.infer(inferencedata)

print(results)
```

This orchestration allows a user to specify the workspace, pipeline, and data connection.  As long as they all match the previous conditions, then the orchestration will run successfully.

### Upload the Orchestration

Orchestrations are uploaded with the Wallaroo client `upload_orchestration(path)` method with the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **path** | string (Required) | The path to the ZIP file to be uploaded. |

Once uploaded, the deployment will be prepared and any requirements will be downloaded and installed.


For this example, the orchestration `./bigquery_remote_inference/bigquery_remote_inference.zip` will be uploaded and saved to the variable `orchestration`.  Then we will loop until the uploaded orchestration's `status` displays `ready`.

In [ ]:
orchestration = wl.upload_orchestration(name="statsmodel-orchestration", path="./forecast-orchestration/forecast-orchestration.zip")

while orchestration.status() != 'ready':
    print(orchestration.status())
    time.sleep(5)

In [83]:
wl.list_orchestrations()

[<wallaroo.orchestration.Orchestration at 0x7fda72ec9910>,
 <wallaroo.orchestration.Orchestration at 0x7fda72e3c2e0>,
 <wallaroo.orchestration.Orchestration at 0x7fda73114bb0>,
 <wallaroo.orchestration.Orchestration at 0x7fda73114af0>,
 <wallaroo.orchestration.Orchestration at 0x7fda73114d90>,
 <wallaroo.orchestration.Orchestration at 0x7fdabddb0e20>,
 <wallaroo.orchestration.Orchestration at 0x7fdabddb0eb0>,
 <wallaroo.orchestration.Orchestration at 0x7fda72e76d60>,
 <wallaroo.orchestration.Orchestration at 0x7fda72e764c0>,
 <wallaroo.orchestration.Orchestration at 0x7fda72e76370>]

### Create the Task

The orchestration is now ready to be implemented as a Wallaroo Task.  We'll just run it once as an example.  This specific Orchestration that creates the Task assumes that the pipeline is deployed, and accepts the arguments:

* workspace_name
* pipeline_name
* bigquery_connection_name

We'll supply the workspaces, pipeline and connection created in previous steps and stored in the initial variables above.  Verify these exist and match the existing workspace, pipeline and connection used in the previous notebooks in this series.

Tasks are generated and run once with the Orchestration `run_once(name, json_args, timeout)` method.  Any arguments for the orchestration are passed in as a `Dict`.  If there are no arguments, then an empty set `{}` is passed.

In [84]:
task = orchestration.run_once(name="statsmodel single run", json_args={"workspace_name":workspace_name, "pipeline_name": pipeline_name, "bigquery_connection_input_name":connection_name})

### Monitor Run with Task Status

We'll monitor the run first with it's status.

For this example, the status of the previously created task will be generated, then looped until it has reached status `started`.

In [85]:
while task.status() != "started":
    display(task.status())
    time.sleep(5)

'pending'

'pending'

### List Tasks

We'll use the Wallaroo client `list_tasks` method to view the tasks currently running.

In [87]:
wl.list_tasks()

[<wallaroo.task.Task at 0x7fda72ff0280>,
 <wallaroo.task.Task at 0x7fda72e9c580>,
 <wallaroo.task.Task at 0x7fda72e6a1c0>,
 <wallaroo.task.Task at 0x7fda73137100>,
 <wallaroo.task.Task at 0x7fda73071c70>,
 <wallaroo.task.Task at 0x7fda73071760>,
 <wallaroo.task.Task at 0x7fda73071430>,
 <wallaroo.task.Task at 0x7fda730716d0>,
 <wallaroo.task.Task at 0x7fda7306f9d0>]

### Display Task Run Results

The Task Run is the implementation of the task - the actual running of the script and it's results.  Tasks that are Run Once will only have one Task Run, while a Task set to Run Scheduled will have a Task Run for each time the task is executed.  Each Task Run has its own set of logs and results that are monitoried through the Task Run `logs()` method.

We'll wait 30 seconds, then retrieve the task run for our generated task, then start checking the logs for our task run.  It may take longer than 30 seconds to launch the task, so be prepared to run the `.logs()` method again to view the logs.

In [90]:
#wait 30 seconds for the task to finish
time.sleep(30)
statsmodel_task_run = task.last_runs()[0]

In [91]:
statsmodel_task_run.logs()

['2023-06-30T16:53:57.629360908Z stdout F statsmodel-bike-rentals-jch',
 '2023-06-30T16:53:57.629426109Z stdout F BigQuery Connection: statsmodel-bike-rentals-jch',
 '2023-06-30T16:53:57.629432309Z stdout F Workspace: multiple-replica-forecast-tutorial-jch',
 '2023-06-30T16:53:57.629438909Z stdout F {\'name\': \'multiple-replica-forecast-tutorial-jch\', \'id\': 7, \'archived\': False, \'created_by\': \'34b86cac-021e-4cf0-aa30-40da7db5a77f\', \'created_at\': \'2023-06-30T15:42:56.551195+00:00\', \'models\': [{\'name\': \'bikedaymodel\', \'versions\': 1, \'owner_id\': \'""\', \'last_update_time\': datetime.datetime(2023, 6, 30, 15, 42, 56, 979723, tzinfo=tzutc()), \'created_at\': datetime.datetime(2023, 6, 30, 15, 42, 56, 979723, tzinfo=tzutc())}], \'pipelines\': [{\'name\': \'bikedaypipe\', \'create_time\': datetime.datetime(2023, 6, 30, 15, 42, 56, 781150, tzinfo=tzutc()), \'definition\': \'[]\'}]}',
 '2023-06-30T16:53:57.629445909Z stdout F Pipeline: bikedaypipe',
 "2023-06-30T16:53:57.629450609Z stdout F {'name': 'bikedaypipe', 'create_time': datetime.datetime(2023, 6, 30, 15, 42, 56, 781150, tzinfo=tzutc()), 'definition': '[]'}",
 '2023-06-30T16:53:57.629455109Z stdout F Getting date and input query.',
 "2023-06-30T16:53:57.629459809Z stdout F Get the current month and retrieve next month's forecasts",
 '2023-06-30T16:53:57.629464109Z stdout F Start date: 7-1-2011',
 "2023-06-30T16:53:57.629469609Z stdout F Forecast dates: ['2011-07-01', '2011-07-08', '2011-07-15', '2011-07-22', '2011-07-29']",
 '2023-06-30T16:53:57.629474009Z stdout F Current date: 2011-07-01',
 '2023-06-30T16:53:57.629478009Z stdout F ',
 '2023-06-30T16:53:57.629483509Z stdout F                 select cnt from release_testing_2023_2.bike_rentals where ',
 "2023-06-30T16:53:57.629487909Z stdout F                 dteday >= DATE_SUB(DATE('2011-07-01'), INTERVAL 1 month) ",
 "2023-06-30T16:53:57.629492209Z stdout F                 AND dteday < DATE('2011-07-01') ",
 '2023-06-30T16:53:57.629496409Z stdout F                 ORDER BY dteday',
 '2023-06-30T16:53:57.629504909Z stdout F Current date: 2011-07-08',
 '2023-06-30T16:53:57.629500609Z stdout F                 ',
 '2023-06-30T16:53:57.629508909Z stdout F ',
 '2023-06-30T16:53:57.629513209Z stdout F                 select cnt from release_testing_2023_2.bike_rentals where ',
 "2023-06-30T16:53:57.629517909Z stdout F                 dteday >= DATE_SUB(DATE('2011-07-08'), INTERVAL 1 month) ",
 '2023-06-30T16:53:57.629527309Z stdout F                 ORDER BY dteday',
 "2023-06-30T16:53:57.629522709Z stdout F                 AND dteday < DATE('2011-07-08') ",
 '2023-06-30T16:53:57.629532009Z stdout F                 ',
 '2023-06-30T16:53:57.629536709Z stdout F Current date: 2011-07-15',
 '2023-06-30T16:53:57.629540709Z stdout F ',
 '2023-06-30T16:53:57.629545309Z stdout F                 select cnt from release_testing_2023_2.bike_rentals where ',
 "2023-06-30T16:53:57.629549809Z stdout F                 dteday >= DATE_SUB(DATE('2011-07-15'), INTERVAL 1 month) ",
 '2023-06-30T16:53:57.629558309Z stdout F                 ORDER BY dteday',
 "2023-06-30T16:53:57.629554009Z stdout F                 AND dteday < DATE('2011-07-15') ",
 '2023-06-30T16:53:57.629562809Z stdout F                 ',
 '2023-06-30T16:53:57.629567509Z stdout F Current date: 2011-07-22',
 '2023-06-30T16:53:57.629571809Z stdout F ',
 '2023-06-30T16:53:57.629576709Z stdout F                 select cnt from release_testing_2023_2.bike_rentals where ',
 "2023-06-30T16:53:57.629581309Z stdout F                 dteday >= DATE_SUB(DATE('2011-07-22'), INTERVAL 1 month) ",
 "2023-06-30T16:53:57.62958591Z stdout F                 AND dteday < DATE('2011-07-22') ",
 '2023-06-30T16:53:57.62959071Z stdout F                 ORDER BY dteday',
 '2023-06-30T16:53:57.62959521Z stdout F                 ',
 '2023-06-30T16:53:57.62959971Z stdout F Current date: 2011-07-29',
 '2023-06-30T16:53:57.62960921Z stdout F                 select cnt from release_testing

### Undeploy the Pipeline

Undeploy the pipeline and return the resources back to the Wallaroo instance.

In [92]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s ..................................... ok


{'name': 'bikedaypipe', 'create_time': datetime.datetime(2023, 6, 30, 15, 42, 56, 781150, tzinfo=tzutc()), 'definition': "[{'ModelInference': {'models': [{'name': 'bikedaymodel', 'version': 'd60ceac2-6fed-4bef-afd1-f3880ad85d0c', 'sha': '525ea2be4402725878382631c2c32b2e3f105bf78eedf41f3ac6d71c0dfa986b'}]}}]"}